## Semantic Web Technology - Data Science Perspective

Set up some simple formatting defaults for the notebook

In [ ]:
%%html
<style>
table {float:left}
</style>

## Overview

This notebook works through the process of extracting data from an Excel spreadsheet, converting the data using the Resource Description Framework (RDF), loading RDF data into a database, semantically enriching the data using ontology languages, and finally demonstrating some flexible querying mechanisms.

### Technologies used in this notebook

Category | Technology | Link
-------- | ---------- | ----
User Interface | Jupyter | [Jupyter](http://jupyter.org) <br />
Raw Data | Excel Spreadsheet | [Excel Description](https://en.wikipedia.org/wiki/Microsoft_Excel) <br />
Database | Virtuoso Open-Source | [Virtuoso GitHub](https://github.com/openlink/virtuoso-opensource) <br />
Resource Description | RDF | [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework) <br />
Ontology Description | RDFS / OWL | [RDFS](https://en.wikipedia.org/wiki/RDF_Schema) / [OWL](https://en.wikipedia.org/wiki/Web_Ontology_Language) <br />
RDF Conversion Utility | csv2rdf | [RDFLib GitHub](https://github.com/RDFLib/rdflib/blob/master/rdflib/tools/csv2rdf.py) <br />
Query Language | SPARQL | [SPARQL](https://en.wikipedia.org/wiki/SPARQL) <br />
Programming Language | Python3 | [Python3](https://www.python.org) <br />
Data Handling | Pandas | [Pandas](http://pandas.pydata.org) <br />
SPARQL Wrapper | SPARQLWrapper | [SPARQLWrapper GitHub](https://github.com/RDFLib/sparqlwrapper) <br />



### Conversion and uploading of data into the database / triple store

Get the filename of the Excel spreadsheet:

In [ ]:
!ls data/communities/*.xlsm

Use pandas to load the spreadsheet and list worksheet names:

In [ ]:
import pandas as pd

In [ ]:
crdata = pd.ExcelFile("Example.xlsm")

In [ ]:
crdata.sheet_names

Two interesting worksheets are 'Gauge' and 'Property' that give a lot of data about gauging stations and properties respectively. We'll use these two worksheets in the rest of the notebook.

Load the gauge worksheet into a pandas dataframe and get overview of column names:

In [ ]:
gauge_df = crdata.parse('Gauge', header=1)

In [ ]:
gauge_df.head()

Load the property worksheet into a pandas dataframe and get overview of column names:

In [ ]:
property_df = crdata.parse('Property')

In [ ]:
property_df.head()

Save the gauge and property worksheets to csv files for subsequent RDF processing:

In [ ]:
gauge_df.to_csv('data/communities/gauge.csv')

In [ ]:
!ls data/communities

In [ ]:
property_df.to_csv('data/communities/property.csv')

In [ ]:
!ls data/communities

We can now convert the csv files to triple format using the csv2rdf utility. RDF namespaces for subject base names and property basenames. The resulting RDF files are in [Turtle](https://www.w3.org/TR/turtle/) syntax.

In [ ]:
# http://ensembleprojects.org/ds/ns/floodrisk/gauge#
# http://ensembleprojects.org/ds/ns/floodrisk/gauge_data#

!python data/communities/csv2rdf.py -b http://ensembleprojects.org/ds/ns/floodrisk/gauge# -p http://ensembleprojects.org/ds/ns/floodrisk/gauge_data# -o data/communities/gauge_ds.ttl data/communities/gauge.csv

We can now take a quick look at the resulting triples and check the namespaces have been generated correctly: 

In [ ]:
!head data/communities/gauge_ds.ttl

Now generate the property triples:

In [ ]:
# http://ensembleprojects.org/ds/ns/floodrisk/property#
# http://ensembleprojects.org/ds/ns/floodrisk/property_data#

!python data/communities/csv2rdf.py -b http://ensembleprojects.org/ds/ns/floodrisk/property# -p http://ensembleprojects.org/ds/ns/floodrisk/property_data# -o data/communities/property_ds.ttl data/communities/property.csv

In [ ]:
!ls data/communities/*.ttl

We can now load the generated files into the Virtuoso database. Virtuoso uses trusted directories for uploading of data, so the ttl files care copied there:

In [ ]:
# Copy ttl files to allowed Virtuoso import directory
!cp data/communities/gauge_ds.ttl /usr/local/Cellar/virtuoso/7.2.4.2/share/virtuoso/vad/
!cp data/communities/property_ds.ttl /usr/local/Cellar/virtuoso/7.2.4.2/share/virtuoso/vad/


We can now load the files into Virtuoso using the isql interface and two small batch files. The triples are loaded into two separate named graphs: (i) <http://ensembleprojects.org/ds/floodrisk/gauge> and (ii) <http://ensembleprojects.org/ds/floodrisk/gauge>:

In [ ]:
!cat data/communities/load_gauge_graph

In [ ]:
!cat data/communities/load_property_graph

In [ ]:
# Load gauge ttl files into Virtuoso named graphs
!isql localhost dba dba data/communities/load_gauge_graph

In [ ]:
# Load property ttl files into Virtuoso named graphs
!isql localhost dba dba data/communities/load_property_graph

### Querying of Semantic Data

Now the raw data has been converted into triple form and uploaded into the Virtuoso triple store, we can query it using the SPARQL language. In this case the SPARQL query is embedded in Python using the SPARQLWrapper package. We have created two separate named graphs so we can query across one or both of them:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Create the SPARQL query as a string. To illustrate querying, we can query across 
# both named graphs using the 'FROM' clause or, as in this case, simply comment out
# one of the named graphs.

sparql_query = """
SELECT ?subject ?predicate ?object
FROM <http://ensembleprojects.org/ds/floodrisk/gauge>
#FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  ?subject ?predicate ?object
}
LIMIT 5
"""

# Virtuoso SPARQL endpoint
sparql_endpoint = "http://localhost:0/sparql"
sparql = SPARQLWrapper(sparql_endpoint)

# Return results in JSON format
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["subject"]["value"], result["predicate"]["value"], result["object"]["value"])

### Semantically enriching the data using an ontology 

The existing property dataset has different damage estimations on a per building basis for a number of different return periods. A simple exmaple of semantically enriching the data is to model these different return periods as a class hierachy; we can then query either specific return periods or all return periods.

The first step is to find the RDF properties that relate to the different damage estimations. To do this, we use a regular expression based filter:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
SELECT DISTINCT ?property
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  ?s ?property ?o .
  FILTER regex(?property,'existingdamage','i')
}
"""

sparql_endpoint = "http://localhost:0/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result['property']['value'])

These look like the correct RDF properites for damage estimations. We can model these as a property hierachy with a top-level 'Q' return period and subsequent sub-properties for individual return periods.

We do this using the RDFS ontology languge. We declare 'Q' as a rdf:Property type and then declare the specific return period properties as a rdfs:subPropertyOf of 'Q'. In Turtle syntax:

Q a rdf:Property . <br />
q2_existingDamageMean rdfs:subPropertyOf Q . <br />
q5_existingDamageMean rdfs:subPropertyOf Q . <br />
... <br />


The SPARQL code below generates the triples representing this ontology and inserts them into the <http://ensembleprojects.org/ds/floodrisk/property> named graph.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
PREFIX g:       <http://ensembleprojects.org/ns/floodrisk/gauge#>
PREFIX gd:      <http://ensembleprojects.org/ns/floodrisk/gauge_data#>
PREFIX p:       <http://ensembleprojects.org/ns/floodrisk/property#>
PREFIX pd:      <http://ensembleprojects.org/ns/floodrisk/property_data#>
PREFIX powl:    <http://ensembleprojects.org/owl/propertymodel#>
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#> 

INSERT {
  powl:Q a rdf:Property . 
  ?p rdfs:subPropertyOf powl:Q 
}
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  ?s ?p ?o .
  FILTER regex(?p,'existingdamage','i')
}
"""

sparql_endpoint = "http://localhost:0/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
# As we're updating the triple store, we need to use the 'POST' method
sparql.setMethod('POST')
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result)

Now the ontology triples are in the database, we need to tell Virtuoso to generate new triples using its inferencing engine. This is done through the Virtuoso isql interface:

$ isql <br />
SQL> rdfs_rule_set('http://ensembleprojects.org/ds/floodrisk/property',  'http://ensembleprojects.org/ds/floodrisk/property'); <br />
SQL>exit; <br />

In Virtuoso, we use the 'DEFINE'statement to give a custom inferencing context. We can then query damage estimations for all return periods using the generic 'Q' return period:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
DEFINE input:inference <http://ensembleprojects.org/ds/floodrisk/property>
PREFIX p:       <http://ensembleprojects.org/ds/ns/floodrisk/property#>
PREFIX powl:    <http://ensembleprojects.org/owl/propertymodel#>

SELECT *
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  p:0 powl:Q ?value .
}
"""
sparql_endpoint = "http://localhost:0/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["value"]['value'])

Although we can get all the return period damage estimations using the above method, in general we want to know both the values and the associated return periods:

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql_query = """
DEFINE input:inference <http://ensembleprojects.org/ds/floodrisk/property>
PREFIX p:       <http://ensembleprojects.org/ds/ns/floodrisk/property#>
PREFIX pd:      <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
PREFIX powl:    <http://ensembleprojects.org/owl/propertymodel#>
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#> 

SELECT ?returnPeriod ?value
FROM <http://ensembleprojects.org/ds/floodrisk/property>
WHERE {
  p:0 ?returnPeriod ?value .
  ?returnPeriod rdfs:subPropertyOf powl:Q
}
"""
sparql_endpoint = "http://localhost:0/sparql"
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setReturnFormat(JSON)

sparql.setQuery(sparql_query)
results = sparql.query().convert()

returnPeriods = []
values = []
for result in results["results"]["bindings"]:
    returnPeriod = int(result['returnPeriod']['value'].split('#')[1].split('_')[0].split('q')[1])
    returnPeriods.append(returnPeriod)
    values.append(result['value']['value'])

vals = zip(returnPeriods, values)
sorted_vals = sorted(vals)
returnPeriods = [val[0] for val in sorted_vals]
values = [val[1] for val in sorted_vals]

print('Period\t\tDamage Estimation')
for i in range(len(returnPeriods)):
    print(returnPeriods[i], '\t\t', values[i])